In [76]:
import pandas as pd
import numpy as np

In [4]:
def read_in_file(model_name):
    filename = f'../results/2_{model_name}_results.csv'
    df = pd.read_csv(filename, on_bad_lines='warn')
    df = df.drop(['Unnamed: 0', 'Template', 'input', 'Ethnicity Probability'], axis=1)
    df = df[df['descriptor'] != 'wear glasses']
    return df

In [39]:
def add_ethnicity_group(df):
    asian = ['CHINESE', 'JAPANESE', 'KOREAN', 'THAI']
    european = ['ENGLISH', 'FRENCH', 'GERMAN', 'GREEK', 'HUNGARIAN', 'ITALIAN', 'NORDIC', 'DUTCH']
    drop = ['AFRICAN', 'ISRAELI', 'TURKISH', 'ROMANIAN', 'BALTIC', 'SLAV', 'INDIAN']
    arab = ['ARAB']
    hispanic = ['HISPANIC']

    df = df[~df['Ethnicity'].isin(drop)]
    df = df[df['Target Ethnicity'] != 'UNKNOWN']

    df['Ethnicity Group'] = df['Ethnicity'].apply(lambda x: 'EUROPEAN' if x in european else 
                                                    'HISPANIC' if x in hispanic else 
                                                    'ARAB' if x in arab else 
                                                    'ASIAN' if x in asian else None)
    return df


In [34]:
def get_proportional_perplexities(df): 
    # Calculate the average perplexity for each group
    cluster_summary = []

    overall_mean = df['Perplexity'].mean()

    for ethnicity in df['Group'].unique():
        cluster_data = df[(df['Group'] == ethnicity)]
        
        # Get the average perplexity for the current group
        cluster_ethnicity = cluster_data['Group'].iloc[0]
        average_perplexity = round(cluster_data['Perplexity'].mean(), 2)
        prop_perplexity = round(cluster_data['Perplexity'].mean() / overall_mean, 3)

        cluster_summary.append({'Group': cluster_ethnicity, 
        'Average Perplexity': average_perplexity, 'Proportional Perplexity': prop_perplexity})

    # Create a DataFrame from the cluster summary list
    prop_df = pd.DataFrame(cluster_summary)

    # Sort the DataFrame by perplexity in ascending order
    prop_df = prop_df.sort_values(by='Average Perplexity')

        # Merge the DataFrames based on 'Group'
    df = pd.merge(df, prop_df, on='Group', how='left')

    # Calculate Adjusted Perplexity
    df['Adjusted Perplexity'] = df['Perplexity'] / df['Proportional Perplexity']
    df = df.drop(['Average Perplexity', 'Proportional Perplexity'], axis=1)

    return df

In [60]:
def get_summary_df(df, apx):
    # Calculate the average perplexity for each cluster and descriptor
    cluster_summary = []

    # Get unique combinations of 'Target Gender', 'Target Ethnicity', and 'descriptor'
    unique_combinations2 = list(itertools.product(df['Target Ethnicity'].unique(), df['descriptor'].unique()))

    for ethnicity in df['Ethnicity Group'].unique():
        for target_ethnicity, descriptor in unique_combinations2:
            # Filter the DataFrame to get rows matching the current combination and group_id
            group_df = df[(df['Ethnicity Group'] == ethnicity) 
                        & (df['Target Ethnicity'] == target_ethnicity) 
                        & (df['descriptor'] == descriptor)]

            # Check if any rows exist for the current combination and group_id
            if not group_df.empty:
                # Get the ethnicity, gender, and average perplexity for the current cluster
                cluster_ethnicity = group_df['Ethnicity Group'].iloc[0]
                if apx:
                    average_perplexity = round(group_df['Adjusted Perplexity'].mean(), 2)
                else:
                    average_perplexity = round(group_df['Perplexity'].mean(), 2)

                cluster_summary.append({'Ethnicity Group': cluster_ethnicity, 'Average Perplexity': average_perplexity, 
                                        'descriptor': descriptor, 'Target Ethnicity': target_ethnicity})

    # Create a DataFrame from the cluster summary list
    summary_df = pd.DataFrame(cluster_summary)

    # Sort the DataFrame by descriptor in ascending order
    summary_df = summary_df.sort_values(by='descriptor')

    return(summary_df)


In [73]:
def mean_reciprocal_rank(rs):
    """
    Score is reciprocal of the rank of the first relevant item
    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).
    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank

    >>> rs = [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.61111111111111105

    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean reciprocal rank

    """
    
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])

In [92]:
def get_mrr_df(df):
    # Calculate the average perplexity for each cluster and descriptor
    rank_array = []
    
    for descriptor in df['descriptor'].unique():
        # Get only rows pertaining to descriptors
        group_df = df[(df['descriptor'] == descriptor)]
        # Rank the average perplexities for each group
        ranked_df = group_df.sort_values(by='Average Perplexity').reset_index(drop=True)
        # Find the index of the row where Target Ethnicity matches
        target_index = ranked_df[ranked_df['Ethnicity Group'] == ranked_df.iloc[0]['Target Ethnicity']].index[0]
        # Create the desired array [0, 1, 0, 0]
        result_array = [int(i == target_index) for i in range(len(ranked_df))]
        rank_array.append(result_array)
    
    mrr = round(mean_reciprocal_rank(rank_array), 3)
    print(mrr)
    return mrr


In [93]:

def print_full_accuracy_table(model_name, apx):
    df = read_in_file(model_name)
    df = add_ethnicity_group(df)
    df = get_proportional_perplexities(df)
    summary_df = get_summary_df(df, apx)
    mrr = get_mrr_df(summary_df)    
    return mrr


In [94]:
model_list = ['roberta_large', 'gpt2_xl', 'microsoft_GODEL_v1_1_large_seq2seq', 'google_flan_ul2', 'EleutherAI_gpt_neox_20b']

for model in model_list:
    print(model)
    print_full_accuracy_table(model, False)
    print_full_accuracy_table(model, True)


roberta_large
0.532
0.652
gpt2_xl
0.578
0.665
microsoft_GODEL_v1_1_large_seq2seq
0.544
0.614
google_flan_ul2
0.572
0.657
EleutherAI_gpt_neox_20b
0.523
0.617


In [50]:
import pandas as pd

# Perplexity data in the format: (Ethnicity, Perplexity)
perplexity_data = [
    ('ARAB', 6, 11),
    ('HISPANIC', 8, 12),
    ('EUROPEAN', 8, 13),
    ('ASIAN', 9, 19)
]

# Create a pandas DataFrame
df = pd.DataFrame(perplexity_data, columns=['Ethnicity', 'Perplexity_Muslim', 'Perplexity_Bearded'])

# Calculate average perplexity for each phrase
avg_perplexity_muslim = df['Perplexity_Muslim'].mean()
avg_perplexity_bearded = df['Perplexity_Bearded'].mean()

# Calculate ratios for each ethnic group
df['Ratio_Muslim'] = df['Perplexity_Muslim'] / avg_perplexity_muslim
df['Ratio_Bearded'] = df['Perplexity_Bearded'] / avg_perplexity_bearded

# Rank stereotypes based on ratios
ranked_muslim = df.sort_values(by='Ratio_Muslim', ascending=True)
ranked_bearded = df.sort_values(by='Ratio_Bearded', ascending=True)

# Print ranked stereotypes
print("Ranking for 'I'm Muslim':")
for rank, row in ranked_muslim.iterrows():
    print(f"{rank + 1}. {row['Ethnicity']}: Ratio = {row['Ratio_Muslim']:.2f}")

print("\nRanking for 'I'm bearded':")
for rank, row in ranked_bearded.iterrows():
    print(f"{rank + 1}. {row['Ethnicity']}: Ratio = {row['Ratio_Bearded']:.2f}")


Ranking for 'I'm Muslim':
1. ARAB: Ratio = 0.77
2. HISPANIC: Ratio = 1.03
3. EUROPEAN: Ratio = 1.03
4. ASIAN: Ratio = 1.16

Ranking for 'I'm bearded':
1. ARAB: Ratio = 0.80
2. HISPANIC: Ratio = 0.87
3. EUROPEAN: Ratio = 0.95
4. ASIAN: Ratio = 1.38


In [51]:
df

,Ethnicity,Perplexity_Muslim,Perplexity_Bearded,Ratio_Muslim,Ratio_Bearded
0,ARAB,6,11,0.774194,0.800000
1,HISPANIC,8,12,1.032258,0.872727
2,EUROPEAN,8,13,1.032258,0.945455
3,ASIAN,9,19,1.161290,1.381818
